In [1]:
import qdrant_client
import os, nest_asyncio
from IPython.display import Markdown, display
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import StorageContext, get_response_synthesizer, VectorStoreIndex, SimpleDirectoryReader, PromptTemplate
os.chdir('/Users/1zuu/Library/Mobile Documents/com~apple~CloudDocs/ML DEV/LLM RESEARCH 2024/LLM in Production/01-diy-llm-qa-bot')
nest_asyncio.apply() 

In [2]:
from util.llm_services import *

Using Groq LLM ....
Using HuggingFace Embedding ....


/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
documents = SimpleDirectoryReader(path_config.kb_documents_dir).load_data()
documents

[Document(id_='c4c5e8e9-a328-4de0-be3e-93115b24170e', embedding=None, metadata={'file_path': '/Users/1zuu/Library/Mobile Documents/com~apple~CloudDocs/ML DEV/LLM RESEARCH 2024/LLM in Production/01-diy-llm-qa-bot/db/kb/Accessing:Customizing Prompts within Higher-Level Modules - LlamaIndex.html', 'file_name': 'Accessing:Customizing Prompts within Higher-Level Modules - LlamaIndex.html', 'file_type': 'text/html', 'file_size': 1241637, 'creation_date': '2024-06-03', 'last_modified_date': '2024-06-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='\n<!doctype html>\n<html lang="en" class="no-js">\n  <head>\n    \n      <meta charset="utf-8">\n      <meta name="viewport" content="width=device-width,initial-scale=1">\n      \n      \n      \n      \n        

In [4]:
# token_count_hnb = len(tiktoken_encoder.encode(document_hnb))
# token_count_peoples = len(tiktoken_encoder.encode(document_peoples))

# print(f"Total token count HNB: {token_count_hnb}")
# print(f"Total token count Peoples Bank: {token_count_peoples}")

In [5]:
nodes = Settings.node_parser.get_nodes_from_documents(documents)
print(f"Total nodes: {len(nodes)}")

Total nodes: 559


In [ ]:
collection_name = "banks"
client = qdrant_client.QdrantClient(
                                path=path_config.vector_store_path
                                )
vector_store = QdrantVectorStore(
                                client=client, 
                                collection_name=collection_name
                                )
if not os.path.exists(f"{path_config.vector_store_path}/collection/{collection_name}"):
    print("Building Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store,
                                                    )
    index = VectorStoreIndex(
                            nodes, 
                            storage_context=storage_context
                            )
    # index.storage_context.vector_store.persist(persist_path=path_config.vector_store_path)
    
else:
    print("Loading Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store
                                                    )
    index = VectorStoreIndex.from_vector_store(
                                                vector_store, 
                                                storage_context=storage_context
                                                )

retriever = VectorIndexRetriever(
                                index=index,
                                similarity_top_k=3,
                                )

response_synthesizer = get_response_synthesizer()
query_engine = RetrieverQueryEngine(
                                    retriever=retriever,
                                    response_synthesizer=response_synthesizer,
                                    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
                                    )

In [ ]:
question = "What are the Eligibility criteria to open a Fixed Deposit?"

In [ ]:
response = query_engine.query(question)
display(Markdown(str(response)))

In [ ]:
prompts_dict = query_engine.get_prompts()
prompts_dict

In [ ]:
## Text QA Template

print(prompts_dict['response_synthesizer:text_qa_template'].default_template.template)

In [ ]:
## Refine QA Template

print(prompts_dict['response_synthesizer:refine_template'].default_template.template)

### Reproduce the Query Engine with new prompt

In [6]:
collection_name = "banks"
client = qdrant_client.QdrantClient(
                                path=path_config.vector_store_path
                                )
vector_store = QdrantVectorStore(
                                client=client, 
                                collection_name=collection_name
                                )
if not os.path.exists(f"{path_config.vector_store_path}/collection/{collection_name}"):
    print("Building Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store,
                                                    )
    index = VectorStoreIndex(
                            nodes, 
                            storage_context=storage_context
                            )
else:
    print("Loading Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store
                                                    )
    index = VectorStoreIndex.from_vector_store(
                                                vector_store, 
                                                storage_context=storage_context
                                                )

retriever = VectorIndexRetriever(
                                index=index,
                                similarity_top_k=3,
                                )

response_synthesizer = get_response_synthesizer()
query_engine = RetrieverQueryEngine(
                                    retriever=retriever,
                                    response_synthesizer=response_synthesizer,
                                    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
                                    )


text_chat_template = """
Context Information is below.
---------------------
{context_str}
---------------------

Chat History and Query is below.
---------------------
{query_str}
---------------------

Given the Context Information, Chat History and not prior knowledge, answer the query.

ASSISTANT: 
"""

text_chat_pmt_template = PromptTemplate(text_chat_template)   

query_engine.update_prompts(
                            {"response_synthesizer:text_qa_template": text_chat_pmt_template}
)

Loading Finance Index ...


In [7]:
prompts_dict = query_engine.get_prompts()
print(prompts_dict['response_synthesizer:text_qa_template'])

metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['context_str', 'query_str'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='\nContext Information is below.\n---------------------\n{context_str}\n---------------------\n\nChat History and Query is below.\n---------------------\n{query_str}\n---------------------\n\nGiven the Context Information, Chat History and not prior knowledge, answer the query.\nAnswer: \n'


In [8]:
query_engine.query("""USER : Hi !\nASSISTANT : Hello! How can I help you today?\nUSER : What are the Eligibility criteria to open a Fixed Deposit?\n""")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Response(response='To open a Fixed Deposit, you must meet the following eligibility criteria:\n\n1. You must be a citizen of Sri Lanka above 18 years of age holding a valid National Identity Card (NIC), a valid Passport, or a Driving License.\n2. Sri Lankan citizens residing temporarily outside Sri Lanka are also eligible.\n3. A person holding dual citizenship can open a Fixed Deposit.\n4. Non-residents who possess a resident visa can also open a Fixed Deposit.\n\nIn addition to individuals, Term Deposit Accounts can be opened by Partnership firms, Private and Public Limited Companies, Specified Associations, Societies, Trusts, either singly or jointly.\n\nIndividuals can open joint Fixed Deposits, and the operation of such joint accounts can be done by a single joint holder if agreed upon in the mandate under operating instructions. If not, it can be operated by more than one individual or jointly as specified in the mandate.\n\nIf you are a Sri Lankan citizen residing outside of Sri 